In [0]:

from pyspark.sql.functions import *
from pyspark.sql import Window
from pyspark.sql.types import *
import re
from pyspark.sql import functions as F
from functools import reduce

# Helper function for safe casting
def safe_cast(column, data_type):
    return F.expr(f"try_cast({column} as {data_type})")

# COMMAND ----------

# MAGIC %md
# MAGIC # Gold Layer Data Quality Checks
# MAGIC ## For master_companies table with LLM enrichment validation

# COMMAND ----------

# MAGIC %md
# MAGIC ## 1. LLM Enrichment Quality Checks

# COMMAND ----------

def check_llm_enrichment_completeness(df):
    """
    Check completeness of LLM-enriched fields
    """
    
    df = df.withColumn(
        "has_llm_industry",
        when(col("industry").isNotNull() & (col("industry") != "Unknown") & (col("industry") != ""), True).otherwise(False)
    )
    
    df = df.withColumn(
        "has_company_size",
        when(col("company_size").isNotNull() & (col("company_size") != "Unknown") & (col("company_size") != ""), True).otherwise(False)
    )
    
    df = df.withColumn(
        "has_products",
        when(col("products_offered").isNotNull() & (length(col("products_offered")) > 0), True).otherwise(False)
    )
    
    df = df.withColumn(
        "has_services",
        when(col("services_offered").isNotNull() & (length(col("services_offered")) > 0), True).otherwise(False)
    )
    
    df = df.withColumn(
        "has_keywords",
        when(col("keywords").isNotNull() & (length(col("keywords")) > 0), True).otherwise(False)
    )
    
    return df

# COMMAND ----------

def check_llm_quality(df):
    """
    Check quality of LLM-generated content
    - Minimum length requirements
    - No generic/placeholder text
    - Proper formatting
    """
    
    # Check if products_offered has meaningful content (not just "N/A" or too short)
    df = df.withColumn(
        "products_quality_good",
        when(
            col("products_offered").isNotNull() &
            (length(col("products_offered")) > 10) &
            ~lower(col("products_offered")).rlike(r"(n/a|not available|unknown|none|null)"),
            True
        ).otherwise(False)
    )
    
    # Check if services_offered has meaningful content
    df = df.withColumn(
        "services_quality_good",
        when(
            col("services_offered").isNotNull() &
            (length(col("services_offered")) > 10) &
            ~lower(col("services_offered")).rlike(r"(n/a|not available|unknown|none|null)"),
            True
        ).otherwise(False)
    )
    
    # Check if keywords are meaningful (multiple keywords separated by comma/semicolon)
    df = df.withColumn(
        "keywords_quality_good",
        when(
            col("keywords").isNotNull() &
            (length(col("keywords")) > 5) &
            (col("keywords").rlike(r"[,;]") | (length(col("keywords")) > 20)),
            True
        ).otherwise(False)
    )
    
    # Check if company_size is from valid categories
    df = df.withColumn(
        "company_size_valid",
        when(
            col("company_size").isin(["Small", "Medium", "Large", "Enterprise", "Startup", "SME"]),
            True
        ).otherwise(False)
    )
    
    # Check if industry is not generic
    df = df.withColumn(
        "industry_quality_good",
        when(
            col("industry").isNotNull() &
            ~col("industry").isin(["Unknown", "Other", "N/A", "General"]) &
            (length(col("industry")) > 3),
            True
        ).otherwise(False)
    )
    
    return df

# COMMAND ----------

def calculate_llm_enrichment_score(df):
    """
    Calculate overall LLM enrichment quality score (0-100)
    """
    
    # Weight different LLM fields
    df = df.withColumn(
        "llm_enrichment_score",
        (
            # Completeness checks (40%)
            when(col("has_llm_industry"), 10).otherwise(0) +
            when(col("has_company_size"), 8).otherwise(0) +
            when(col("has_products"), 7).otherwise(0) +
            when(col("has_services"), 7).otherwise(0) +
            when(col("has_keywords"), 8).otherwise(0) +
            
            # Quality checks (60%)
            when(col("industry_quality_good"), 15).otherwise(0) +
            when(col("company_size_valid"), 10).otherwise(0) +
            when(col("products_quality_good"), 15).otherwise(0) +
            when(col("services_quality_good"), 15).otherwise(0) +
            when(col("keywords_quality_good"), 5).otherwise(0)
        )
    )
    
    return df

# COMMAND ----------

# MAGIC %md
# MAGIC ## 2. Gold Layer Data Completeness

# COMMAND ----------

def calculate_gold_completeness_score(df):
    """
    Calculate completeness score for gold layer master_companies table
    Includes both source data and LLM-enriched fields
    """
    
    # String fields
    string_fields = [
        "company_name", "website", "linkedin", "facebook", "instagram",
        "industry", "company_size", "products_offered", "services_offered",
        "keywords", "hq_country"
    ]
    
    # Numeric fields (can be empty strings)
    numeric_fields = [
        "revenue", "founding_year", "no_of_locations_in_singapore"
    ]
    
    total_fields = len(string_fields) + len(numeric_fields)
    
    # Build column expressions for string fields
    string_cols = [F.when(F.col(c).isNotNull() & (F.col(c) != ""), F.lit(1)).otherwise(F.lit(0)) for c in string_fields]
    
    # Build column expressions for numeric fields (handle empty strings)
    numeric_cols = [F.when(
        F.col(c).isNotNull() &
        (F.col(c) != "") &
        F.expr(f"try_cast(`{c}` as double) is not null"),
        F.lit(1)
    ).otherwise(F.lit(0)) for c in numeric_fields]
    
    # Combine all checks
    all_cols = string_cols + numeric_cols
    
    # Sum all completeness checks
    if len(all_cols) > 1:
        completeness_expr = reduce(lambda a, b: a + b, all_cols)
    else:
        completeness_expr = all_cols[0]
    
    # Calculate percentage completeness
    df = df.withColumn(
        "data_completeness_score",
        (completeness_expr / F.lit(total_fields)) * 100
    )
    
    return df

# COMMAND ----------

# MAGIC %md
# MAGIC ## 3. Stock Data Quality Checks

# COMMAND ----------

def check_stock_data_quality(df):
    """
    Validate stock-related data in master_companies
    """
    
    # Check if stock data is consistent
    df = df.withColumn(
        "stock_data_complete",
        when(
            col("stock_exchange_code").isNotNull() &
            col("revenue").isNotNull() &
            col("is_it_delisted").isNotNull(),
            True
        ).otherwise(False)
    )
    
    # Check if revenue is positive (if present) - handle empty strings
    df = df.withColumn(
        "revenue_valid",
        when(
            col("revenue").isNull() |
            (col("revenue") == ""),
            True
        ).when(
            expr("try_cast(revenue as double) is null"),
            False
        ).when(
            expr("try_cast(revenue as double) >= 0"),
            True
        ).otherwise(False)
    )
    
    # Check if founding_year is reasonable (between 1800 and current year) - handle empty strings
    df = df.withColumn(
        "founding_year_valid",
        when(
            col("founding_year").isNull() |
            (col("founding_year") == ""),
            True
        ).when(
            expr("try_cast(founding_year as int) is null"),
            False
        ).when(
            expr("try_cast(founding_year as int) >= 1800 and try_cast(founding_year as int) <= year(current_date())"),
            True
        ).otherwise(False)
    )
    
    # Check number of employees is positive (if present) - handle empty strings
    df = df.withColumn(
        "employees_valid",
        when(
            col("number_of_employees").isNull() |
            (col("number_of_employees") == ""),
            True
        ).when(
            expr("try_cast(number_of_employees as int) is null"),
            False
        ).when(
            expr("try_cast(number_of_employees as int) > 0"),
            True
        ).otherwise(False)
    )
    
    return df

# COMMAND ----------

# MAGIC %md
# MAGIC ## 4. Social Media & Website Quality

# COMMAND ----------

def check_url_validity(df, url_col="website"):
    """
    Check if URL is valid format
    """
    
    return df.withColumn(
        f"{url_col}_is_valid",
        when(
            col(url_col).isNotNull() &
            (col(url_col).rlike(r'^[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,}') |
             col(url_col).rlike(r'^https?://[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,}')),
            True
        ).otherwise(False)
    )

def check_social_media_presence(df):
    """
    Check if company has social media presence
    """
    
    df = df.withColumn(
        "has_social_media",
        when(
            col("linkedin").isNotNull() |
            col("facebook").isNotNull() |
            col("instagram").isNotNull(),
            True
        ).otherwise(False)
    )
    
    df = df.withColumn(
        "social_media_count",
        when(col("linkedin").isNotNull(), 1).otherwise(0) +
        when(col("facebook").isNotNull(), 1).otherwise(0) +
        when(col("instagram").isNotNull(), 1).otherwise(0)
    )
    
    return df

# COMMAND ----------

# MAGIC %md
# MAGIC ## 5. Master Data Quality Function

# COMMAND ----------

def run_all_gold_quality_checks(df):
    """
    Run all quality checks on gold.master_companies table
    """
    
    # LLM enrichment checks
    df = check_llm_enrichment_completeness(df)
    df = check_llm_quality(df)
    df = calculate_llm_enrichment_score(df)
    
    # General completeness
    df = calculate_gold_completeness_score(df)
    
    # Stock data quality
    df = check_stock_data_quality(df)
    
    # Social media and website
    df = check_url_validity(df, "website")
    df = check_social_media_presence(df)
    
    # Calculate overall gold quality score
    df = df.withColumn(
        "overall_gold_quality_score",
        (
            # LLM enrichment quality (50%)
            (col("llm_enrichment_score") * 0.5) +
            
            # Data completeness (30%)
            (col("data_completeness_score") * 0.3) +
            
            # Data validity (20%)
            (
                (
                    when(col("revenue_valid"), 5).otherwise(0) +
                    when(col("founding_year_valid"), 5).otherwise(0) +
                    when(col("employees_valid"), 5).otherwise(0) +
                    when(col("website_is_valid"), 5).otherwise(0)
                )
            )
        )
    )
    
    return df

# COMMAND ----------

# MAGIC %md
# MAGIC ## 6. Quality Report Generation

# COMMAND ----------

def generate_gold_quality_report(df):
    """
    Generate comprehensive quality report for gold layer
    """
    
    total_count = df.count()
    
    report = df.select(
        # Totals
        lit(total_count).alias("total_records"),
        countDistinct("uen").alias("unique_companies"),
        
        # LLM Enrichment Coverage
        (sum(when(col("has_llm_industry"), 1).otherwise(0)) / total_count * 100).alias("llm_industry_coverage_%"),
        (sum(when(col("has_company_size"), 1).otherwise(0)) / total_count * 100).alias("company_size_coverage_%"),
        (sum(when(col("has_products"), 1).otherwise(0)) / total_count * 100).alias("products_coverage_%"),
        (sum(when(col("has_services"), 1).otherwise(0)) / total_count * 100).alias("services_coverage_%"),
        (sum(when(col("has_keywords"), 1).otherwise(0)) / total_count * 100).alias("keywords_coverage_%"),
        
        # LLM Quality
        (sum(when(col("products_quality_good"), 1).otherwise(0)) / total_count * 100).alias("products_quality_%"),
        (sum(when(col("services_quality_good"), 1).otherwise(0)) / total_count * 100).alias("services_quality_%"),
        (sum(when(col("keywords_quality_good"), 1).otherwise(0)) / total_count * 100).alias("keywords_quality_%"),
        
        # Stock Data (handle empty strings)
        (count(when(col("stock_exchange_code").isNotNull() & (col("stock_exchange_code") != ""), 1)) / total_count * 100).alias("stock_data_coverage_%"),
        (count(when(col("revenue").isNotNull() & (col("revenue") != ""), 1)) / total_count * 100).alias("revenue_coverage_%"),
        
        # Social Media
        (sum(when(col("has_social_media"), 1).otherwise(0)) / total_count * 100).alias("social_media_coverage_%"),
        avg("social_media_count").alias("avg_social_platforms_per_company"),
        
        # Overall Scores
        avg("llm_enrichment_score").alias("avg_llm_enrichment_score"),
        avg("data_completeness_score").alias("avg_completeness_score"),
        avg("overall_gold_quality_score").alias("avg_overall_quality_score")
    )
    
    return report

# COMMAND ----------

def export_gold_quality_report(df, output_path):
    """
    Export detailed quality report with all quality flags
    """
    
    detailed_report = df.select(
        "uen",
        "company_name",
        "industry",
        "company_size",
        
        # Scores
        "llm_enrichment_score",
        "data_completeness_score",
        "overall_gold_quality_score",
        
        # LLM Flags
        "has_llm_industry",
        "has_company_size",
        "has_products",
        "has_services",
        "has_keywords",
        
        # Quality Flags
        "products_quality_good",
        "services_quality_good",
        "keywords_quality_good",
        
        # Data Validity
        "revenue_valid",
        "founding_year_valid",
        "website_is_valid",
        "has_social_media"
    ).orderBy(col("overall_gold_quality_score").desc())
    
    # Save report
    detailed_report.coalesce(1).write \
        .format("csv") \
        .option("header", "true") \
        .mode("overwrite") \
        .save(f"{output_path}/gold_quality_report")
    
    print(f"✓ Gold quality report saved to: {output_path}/gold_quality_report")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 7. LLM Enrichment Gap Analysis

# COMMAND ----------

def identify_enrichment_gaps(df):
    """
    Identify companies that need better LLM enrichment
    Returns companies with low LLM enrichment scores
    """
    
    gaps = df.filter(
        (col("llm_enrichment_score") < 50) |
        ~col("has_llm_industry") |
        ~col("has_company_size")
    ).select(
        "uen",
        "company_name",
        "industry",
        "company_size",
        "llm_enrichment_score",
        "has_llm_industry",
        "has_company_size",
        "has_products",
        "has_services",
        "has_keywords"
    ).orderBy("llm_enrichment_score")
    
    return gaps

def export_enrichment_gaps(gaps_df, output_path):
    """
    Export companies needing re-enrichment
    """
    
    gaps_df.coalesce(1).write \
        .format("csv") \
        .option("header", "true") \
        .mode("overwrite") \
        .save(f"{output_path}/enrichment_gaps")
    
    gap_count = gaps_df.count()
    print(f"✓ Found {gap_count} companies needing enrichment improvement")
    print(f"✓ Gap analysis saved to: {output_path}/enrichment_gaps")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 8. Data Coverage Analysis

# COMMAND ----------

def calculate_field_coverage(df):
    """
    Calculate coverage percentage for each important field
    Returns detailed coverage statistics
    """
    
    total_count = df.count()
    
    coverage_stats = df.select(
        lit(total_count).alias("total_companies"),
        
        # Basic Information
        (count(when(col("uen").isNotNull(), 1)) / total_count * 100).alias("uen_coverage_%"),
        (count(when(col("company_name").isNotNull(), 1)) / total_count * 100).alias("company_name_coverage_%"),
        
        # Website & Contact
        (count(when(col("website").isNotNull() & (col("website") != ""), 1)) / total_count * 100).alias("website_coverage_%"),
        (count(when(col("email").isNotNull(), 1)) / total_count * 100).alias("email_coverage_%"),
        (count(when(col("phone").isNotNull(), 1)) / total_count * 100).alias("phone_coverage_%"),
        
        # Social Media
        (count(when(col("linkedin").isNotNull(), 1)) / total_count * 100).alias("linkedin_coverage_%"),
        (count(when(col("facebook").isNotNull(), 1)) / total_count * 100).alias("facebook_coverage_%"),
        (count(when(col("instagram").isNotNull(), 1)) / total_count * 100).alias("instagram_coverage_%"),
        
        # Business Data
        (count(when(col("industry").isNotNull() & (col("industry") != "Unknown"), 1)) / total_count * 100).alias("industry_coverage_%"),
        (count(when(col("company_size").isNotNull(), 1)) / total_count * 100).alias("company_size_coverage_%"),
        (count(when(col("revenue").isNotNull(), 1)) / total_count * 100).alias("revenue_coverage_%"),
        (count(when(col("founding_year").isNotNull(), 1)) / total_count * 100).alias("founding_year_coverage_%"),
        (count(when(col("number_of_employees").isNotNull(), 1)) / total_count * 100).alias("employees_coverage_%"),
        
        # LLM Enriched Fields
        (count(when(col("products_offered").isNotNull() & (length(col("products_offered")) > 0), 1)) / total_count * 100).alias("products_coverage_%"),
        (count(when(col("services_offered").isNotNull() & (length(col("services_offered")) > 0), 1)) / total_count * 100).alias("services_coverage_%"),
        (count(when(col("keywords").isNotNull() & (length(col("keywords")) > 0), 1)) / total_count * 100).alias("keywords_coverage_%"),
        
        # Stock Data
        (count(when(col("stock_exchange_code").isNotNull(), 1)) / total_count * 100).alias("stock_code_coverage_%"),
        
        # Location
        (count(when(col("hq_country").isNotNull(), 1)) / total_count * 100).alias("hq_country_coverage_%"),
        (count(when(col("no_of_locations_in_singapore").isNotNull(), 1)) / total_count * 100).alias("sg_locations_coverage_%")
    )
    
    return coverage_stats

# COMMAND ----------

def export_coverage_report(coverage_df, output_path):
    """
    Export coverage statistics as CSV
    """
    
    coverage_df.coalesce(1).write \
        .format("csv") \
        .option("header", "true") \
        .mode("overwrite") \
        .save(f"{output_path}/data_coverage_report")
    
    print(f"✓ Coverage report saved to: {output_path}/data_coverage_report")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 9. Top Industries Analysis

# COMMAND ----------

def get_top_industries(df, top_n=5):
    """
    Get top N industries by company count
    """
    
    total_companies = df.count()
    
    top_industries = df.filter(
        col("industry").isNotNull() &
        (col("industry") != "Unknown") &
        (col("industry") != "")
    ).groupBy("industry").agg(
        count("*").alias("company_count")
    ).withColumn(
        "percentage_of_total",
        (col("company_count") / lit(total_companies) * 100)
    ).orderBy(
        col("company_count").desc()
    ).limit(top_n)
    
    return top_industries

def get_industry_statistics(df):
    """
    Get comprehensive industry statistics
    """
    
    # Create temp column with revenue as double (handle empty strings)
    df_with_revenue = df.withColumn(
        "revenue_double",
        expr("try_cast(revenue as double)")
    )
    
    industry_stats = df_with_revenue.filter(
        col("industry").isNotNull() &
        (col("industry") != "Unknown")
    ).groupBy("industry").agg(
        count("*").alias("company_count"),
        
        # Coverage metrics
        (count(when(col("website").isNotNull(), 1)) / count("*") * 100).alias("website_coverage_%"),
        (count(when(col("linkedin").isNotNull(), 1)) / count("*") * 100).alias("linkedin_coverage_%"),
        (count(when(col("revenue_double").isNotNull(), 1)) / count("*") * 100).alias("revenue_coverage_%"),
        
        # Quality metrics
        avg("data_completeness_score").alias("avg_completeness_score"),
        avg("llm_enrichment_score").alias("avg_llm_score"),
        
        # Business metrics (use revenue_double which handles empty strings)
        avg("revenue_double").alias("avg_revenue"),
        count(when(col("stock_exchange_code").isNotNull(), 1)).alias("listed_companies_count")
    ).orderBy(col("company_count").desc())
    
    return industry_stats

def export_industry_analysis(industry_df, output_path):
    """
    Export industry analysis
    """
    
    industry_df.coalesce(1).write \
        .format("csv") \
        .option("header", "true") \
        .mode("overwrite") \
        .save(f"{output_path}/industry_analysis")
    
    print(f"✓ Industry analysis saved to: {output_path}/industry_analysis")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 10. Production Monitoring Functions

# COMMAND ----------

def continuous_quality_monitoring(df, baseline_metrics=None):
    """
    Monitor quality metrics vs baseline for production
    Returns quality metrics with alerts if they deviate from baseline
    """
    
    current_metrics = df.select(
        avg("overall_gold_quality_score").alias("avg_quality_score"),
        avg("llm_enrichment_score").alias("avg_llm_score"),
        avg("data_completeness_score").alias("avg_completeness_score"),
        (count(when(col("website").isNotNull(), 1)) / count("*") * 100).alias("website_coverage"),
        (count(when(col("linkedin").isNotNull(), 1)) / count("*") * 100).alias("linkedin_coverage")
    ).collect()[0]
    
    if baseline_metrics:
        # Compare with baseline (deviation > 5% triggers alert)
        alerts = []
        threshold = 5
        
        for metric_name in ["avg_quality_score", "avg_llm_score", "avg_completeness_score"]:
            current_val = current_metrics[metric_name]
            baseline_val = baseline_metrics.get(metric_name)
            
            if baseline_val and abs(current_val - baseline_val) > threshold:
                alerts.append({
                    "metric": metric_name,
                    "current": current_val,
                    "baseline": baseline_val,
                    "deviation": current_val - baseline_val
                })
        
        return current_metrics, alerts
    
    return current_metrics, []

def detect_data_anomalies(df):
    """
    Detect potential data quality anomalies
    """
    
    anomalies = df.select(
        # Duplicate UENs
        countDistinct("uen").alias("unique_uens"),
        count("*").alias("total_records"),
        
        # Invalid data counts
        sum(when(~col("revenue_valid"), 1).otherwise(0)).alias("invalid_revenue_count"),
        sum(when(~col("founding_year_valid"), 1).otherwise(0)).alias("invalid_founding_year_count"),
        sum(when(~col("website_is_valid") & col("website").isNotNull(), 1).otherwise(0)).alias("invalid_website_count"),
        
        # LLM enrichment failures
        sum(when(col("llm_enrichment_score") < 20, 1).otherwise(0)).alias("poor_llm_enrichment_count"),
        
        # Completeness issues
        sum(when(col("data_completeness_score") < 30, 1).otherwise(0)).alias("low_completeness_count")
    )
    
    return anomalies

# COMMAND ----------

# MAGIC %md
# MAGIC ## Usage Example
# MAGIC
# MAGIC ```python
# MAGIC # Read gold master_companies table
# MAGIC master_df = spark.table("gold.master_companies")
# MAGIC
# MAGIC # Run all quality checks
# MAGIC master_df_with_quality = run_all_gold_quality_checks(master_df)
# MAGIC
# MAGIC # ========== DATA QUALITY & ACCURACY TESTING ==========
# MAGIC
# MAGIC # 1. Generate summary quality report
# MAGIC quality_summary = generate_gold_quality_report(master_df_with_quality)
# MAGIC display(quality_summary)
# MAGIC
# MAGIC # 2. Export detailed report
# MAGIC export_gold_quality_report(master_df_with_quality, "abfss://gold@storage.dfs.core.windows.net/reports")
# MAGIC
# MAGIC # 3. Find enrichment gaps
# MAGIC gaps = identify_enrichment_gaps(master_df_with_quality)
# MAGIC display(gaps)
# MAGIC export_enrichment_gaps(gaps, "abfss://gold@storage.dfs.core.windows.net/reports")
# MAGIC
# MAGIC # ========== DATA COVERAGE ANALYSIS ==========
# MAGIC
# MAGIC # Calculate field coverage
# MAGIC coverage = calculate_field_coverage(master_df)
# MAGIC display(coverage)
# MAGIC export_coverage_report(coverage, "abfss://gold@storage.dfs.core.windows.net/reports")
# MAGIC
# MAGIC # ========== TOP 5 INDUSTRIES ==========
# MAGIC
# MAGIC # Get top 5 industries
# MAGIC top_5_industries = get_top_industries(master_df_with_quality, top_n=5)
# MAGIC display(top_5_industries)
# MAGIC
# MAGIC # Get detailed industry statistics
# MAGIC industry_stats = get_industry_statistics(master_df_with_quality)
# MAGIC display(industry_stats)
# MAGIC export_industry_analysis(industry_stats, "abfss://gold@storage.dfs.core.windows.net/reports")
# MAGIC
# MAGIC # ========== PRODUCTION MONITORING ==========
# MAGIC
# MAGIC # Detect anomalies
# MAGIC anomalies = detect_data_anomalies(master_df_with_quality)
# MAGIC display(anomalies)
# MAGIC
# MAGIC # Continuous monitoring (compare with baseline)
# MAGIC baseline = {"avg_quality_score": 75, "avg_llm_score": 70, "avg_completeness_score": 65}
# MAGIC current_metrics, alerts = continuous_quality_monitoring(master_df_with_quality, baseline)
# MAGIC if alerts:
# MAGIC     print("⚠️ Quality Alerts:")
# MAGIC     for alert in alerts:
# MAGIC         print(f"  {alert['metric']}: {alert['deviation']:+.2f}% deviation")
# MAGIC ```

